# Evaluating Capymoa Drift Detectors

This notebook evaluates the impact of a single Concept Drift Detector on a Streaming Machine Learning model on a specified dataset.  
It measures performance in terms of accuracy, execution time, memory usage, and CPU utilization and saves them in a .csv file for benchmarking

## Install and Import Libraries

In [ ]:
%%capture
!pip install capymoa
!pip install memory_profiler

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time
import psutil
import threading
from memory_profiler import memory_usage

### CAPYMOA

from capymoa.classifier import NaiveBayes, HoeffdingTree

from capymoa.datasets import Electricity, Covtype,  Hyper100k, Sensor
from capymoa.datasets import ElectricityTiny # For testing

from capymoa.drift.detectors import ADWIN, STEPD, CUSUM, PageHinkley, DDM, HDDMAverage, HDDMWeighted

from capymoa.evaluation import ClassificationEvaluator, ClassificationWindowedEvaluator
from capymoa.evaluation.results import PrequentialResults
from capymoa.evaluation.visualization import plot_windowed_results

## Evaluation

In [ ]:
# @title Creating Factories
class ClassifierFactory:

    classifier_classes = {
        "NaiveBayes": NaiveBayes,
        "HoeffdingTree": HoeffdingTree
    }

    @staticmethod
    def create(classifier_type, schema):
        """Create a new classifier instance based on type."""


        if classifier_type not in ClassifierFactory.classifier_classes:
            raise ValueError(f"Unknown classifier type: {classifier_type}")

        return ClassifierFactory.classifier_classes[classifier_type](schema=schema)


class StreamFactory:

    stream_classes = {
        "Covtype": Covtype,
        "Electricity": Electricity,
        "Hyper100k": Hyper100k,
        "Sensor": Sensor,
        "ElectricityTiny": ElectricityTiny
    }

    @staticmethod
    def create(stream_type):
        """Create a new stream instance based on type."""


        if stream_type not in StreamFactory.stream_classes:
            raise ValueError(f"Unknown stream type: {stream_type}")

        # Return a new instance of the requested stream
        return StreamFactory.stream_classes[stream_type]()


class DetectorFactory:


    detector_classes = {
        "ADWIN": ADWIN,
        "STEPD": STEPD,
        "CUSUM": CUSUM,
        "PageHinkley": PageHinkley,
        "DDM": DDM,
        "HDDMAverage": HDDMAverage,
        "HDDMWeighted": HDDMWeighted
    }

    @staticmethod
    def create(detector_type):
        """Create a new detector instance based on type."""


        if detector_type == "None":
            return None

        if detector_type not in DetectorFactory.detector_classes:
            raise ValueError(f"Unknown detector type: {detector_type}")

        # Return a new instance of the requested detector
        return DetectorFactory.detector_classes[detector_type]()


In [ ]:
%%capture

# @title Select Parameters

stream = "Sensor" # @param ["Covtype", "Electricity", "Hyper100k", "Sensor", "ElectricityTiny"]

classifier = "NaiveBayes" # @param ["NaiveBayes", "HoeffdingTree"]

detector = "HDDMWeighted" # @param ["ADWIN", "STEPD", "CUSUM", "PageHinkley", "DDM", "HDDMAverage", "HDDMWeighted", "None"]


stream = StreamFactory.create(stream)
classifier = ClassifierFactory.create(classifier, stream.get_schema())
detector = DetectorFactory.create(detector)

WINDOW_SIZE = stream._length // 100 # 1% of dataset size


In [ ]:
def evaluate_detector(detector, stream, classifier):
    i = 0
    cumulative_evaluator = ClassificationEvaluator(schema=stream.get_schema())
    windowed_evaluator = ClassificationWindowedEvaluator(schema=stream.get_schema(), window_size=WINDOW_SIZE)

    # [NOTE] in capymoa==0.9.0, the add_element() method of HDDM_Weighted behaves differently to the other detectors,
    # in this evaluation function the changes are managed autonomously
    changes = []

    while stream.has_more_instances():
        i += 1

        instance = stream.next_instance()

        y = instance.y_index
        y_pred = classifier.predict(instance)

        cumulative_evaluator.update(y, y_pred)
        windowed_evaluator.update(y, y_pred)

        classifier.train(instance)



        if detector != None:
            detector.add_element(y)
            if detector.detected_change():
              # print("Change detected at index: " + str(i))
              classifier = ClassifierFactory.create(classifier.__class__.__name__, stream.get_schema())
              if detector.__class__.__name__ == "HDDMWeighted":
                  changes.append(i)


    if detector.__class__.__name__ == "HDDMWeighted":
        detector.detection_index = changes



    results = PrequentialResults(learner=str(classifier),
                                 stream=stream,
                                 cumulative_evaluator=cumulative_evaluator,
                                 windowed_evaluator=windowed_evaluator)
    return results


In [ ]:
def benchmark_detector(detector, stream, classifier, print_results=False, save_results=True, filename = "results.csv"):

    stream.restart()

    cpu_samples = []

    def monitor_cpu(process, interval=0.1):
        while not stop_event.is_set():
            cpu_samples.append(process.cpu_percent(interval=None))
            time.sleep(interval)

    process = psutil.Process(os.getpid())
    stop_event = threading.Event()
    monitor_thread = threading.Thread(target=monitor_cpu, args=(process,))
    monitor_thread.start()

    start_time = time.time()
    mem_usage, results = memory_usage((evaluate_detector, (detector, stream, classifier)), retval=True)
    end_time = time.time()

    stop_event.set()
    monitor_thread.join()

    cpu_usage = sum(cpu_samples) / len(cpu_samples) / psutil.cpu_count()
    execution_time = end_time - start_time
    max_mem_usage = max(mem_usage)

    results = pd.DataFrame([{
        "dataset": stream.__class__.__name__,
        "classifier": classifier.__class__.__name__,
        "detector": detector.__class__.__name__ if detector else "None",
        "cumulative_accuracy": results.cumulative.metrics_dict()["accuracy"],
        "cumulative_f1": results.cumulative.metrics_dict()["f1_score"],
        "windowed_accuracy": results.windowed.metrics_per_window()["accuracy"].tolist(),
        "windowed_f1": results.windowed.metrics_per_window()["f1_score"].tolist(),
        "execution_time": execution_time,
        "cpu_usage": cpu_usage,
        "memory_usage": max_mem_usage,
        "num_changes": len(detector.detection_index if detector != None else []),
    }])

    if save_results:
        results.to_csv(filename, mode="a", header=not pd.io.common.file_exists(filename), index=False)
        print(f"Results saved to {filename}")
    if print_results:
        print("Results:")
        print(results)

    return results



In [ ]:
benchmark_detector(detector, stream, classifier, save_results=False)

,dataset,classifier,detector,cumulative_accuracy,cumulative_f1,windowed_accuracy,windowed_f1,execution_time,cpu_usage,memory_usage,num_changes
0,Sensor,NaiveBayes,HDDMWeighted,2.12199,NaN,"[2.0, 2.0, 2.0, 4.0, 1.0, 1.0, 3.0, 2.0, 2.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",533.179704,49.998079,1325.832031,515786
